In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [5]:
# 1
df = pd.read_parquet('./data/yellow_tripdata_2022-01.parquet')
print(len(df.columns))
print(df.columns)

19
Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')


In [12]:
# 2
df['duration'] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
df['duration'] = df.duration.apply(lambda td: td.total_seconds() / 60)

print(round(df['duration'].std(), 2))

46.45


In [22]:
# 3
print(round((df[(df.duration >= 1) & (df.duration <= 60)].shape[0] / df.shape[0]) * 100))

df = df[(df.duration >= 1) & (df.duration <= 60)]

98


In [27]:
# 4
categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)

train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

print(len(dv.feature_names_))

/tmp/ipykernel_823006/1413080426.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)


515


In [29]:
# 5
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

print(round(mean_squared_error(y_train, y_pred, squared=False), 2))

6.99


In [40]:
# 6
df = pd.read_parquet('./data/yellow_tripdata_2022-02.parquet')

df['duration'] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
df['duration'] = df.duration.apply(lambda td: td.total_seconds() / 60)

df = df[(df.duration >= 1) & (df.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']

df[categorical] = df[categorical].astype(str)
val_dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

target = 'duration'
y_val = df[target].values

y_pred_val = lr.predict(X_val)

print(round(mean_squared_error(y_val, y_pred_val, squared=False), 2))

7.79


## Check always is correct

In [32]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [34]:
df_train = read_dataframe('./data/yellow_tripdata_2022-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2022-02.parquet')

In [35]:
len(df_train), len(df_val)

(2421440, 2918187)

In [36]:
categorical = ['PULocationID', 'DOLocationID']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [37]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [39]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_train = lr.predict(X_train)
y_pred_val = lr.predict(X_val)

print(round(mean_squared_error(y_train, y_pred_train, squared=False), 2))
print(round(mean_squared_error(y_val, y_pred_val, squared=False), 2))

6.99
7.79
